In [9]:

# # import building polygons and road network from GIS data

# # create a centroid for each building polygon
# def centroid(x, y):
#     # sum of all x coordinates divided by number of x coordinates
#     # sum of all y coordinates divided by number of y coordinates
#     x = np.array(x)
#     y = np.array(y)
#     centroid_x = np.sum(x)/len(x)
#     centroid_y = np.sum(y)/len(y)
#     return centroid_x, centroid_y

# # create a building class with multiple attributes
# class Building:
#     def __init__(self, id, x_coordinate, y_coordinate, structure_type, functionality, plan_area, height, age):
#         self.id = id    # from GIS data
#         self.x_coordinate = x_coordinate    # from centroid_x
#         self.y_coordinate = y_coordinate    # from centroid_y
#         self.structure_type = structure_type
#         self.functionality = functionality
#         self.plan_area = plan_area  # ground cover in square meters
#         self.height = height
#         self.age = age  # from completion date

#     # find building damage states
#     def damage_state(self, damage_state):
#         self.damage_state = damage_state

# # calculate estimated number of people in each building
# # calculate estimated number of trapped people
# # define severity levels
# # calculate number of people in each severity level
# # determine type of actions required for each building

# # create a rescue team class with multiple attributes
# class RescueTeam:
#     def __init__(self, id, x_coordinate, y_coordinate, main_team, skill_level, allocation_type, capacity):
#         self.id = id
#         self.main_team = main_team
#         self.x_coordinate = x_coordinate
#         self.y_coordinate = y_coordinate 
#         self.skill_level = skill_level
#         self.allocation_type = allocation_type  # preemption type (full, partial, none)
#         self.capacity = capacity    # number of people it can rescue
    
#     # create sub class of individual rescue team members
#     class Member:
#         def __init__(self, id, skill_level, capacity):
#             self.id = id
#             self.skill_level = skill_level
#             self.capacity = capacity

# # add rescue team members to rescue team
# member_1 = RescueTeam.Member(1, 1, 1)
# member_2 = RescueTeam.Member(2, 1, 1)
# # matrix of rescue team with SAR actions for each damage state, severity level, and building type

# # create a flow graph for each rescue team (sink, source, edges)
# def flow_graph():
#     # source node
#     # sink node
#     # edges

#     # return flow graph   
#     pass

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

class Area:
    def __init__(self, area_id, name):
        self.area_id = area_id
        self.name = name
        self.sub_areas = []

    def add_sub_area(self, sub_area):
        self.sub_areas.append(sub_area)

class Sub_Area:
    def __init__(self, sub_area_id, name):
        self.sub_area_id = sub_area_id
        self.name = name
        self.buildings = [] 
        self.area = None

    def add_building(self, building):
        self.buildings.append(building)
        building.sub_areas = self
        building.area = self.area

class Building:
    def __init__(self, building_id, x_coordinate, y_coordinate, damage_state, structure_type, functionality, occupant_estimation ,plan_area, height, floor_amount, age):
        self.building_id = building_id                                      # retrieve from osm 
        # self.geometry = None                                              # retrieve from osm 
        self.x_coordinate = x_coordinate                                    # retrieve from osm: centroid_x
        self.y_coordinate = y_coordinate                                    # retrieve from osm: centroid_y
        self.damage_state = damage_state                                    # generate according to theory
        self.structure_type = structure_type                                # generate "fake data" by: assigning structure types whilst regarding clustering
        self.functionality = functionality                                  # generate "fake data" by: assigning functionalities whilst regarding clustering
        self.occupant_estimation = occupant_estimation                      # generate according to theory
        self.plan_area = plan_area                                          # retrieve from osm 
        self.height = height                                                # generate "fake data" by: randomly assigning within a domain
        self.floor_amount = int(math.ceil(self.height / 3.0))               # generate "fake data" by: deviding height by 3.0 as in integer
        self.age = age                                                      # generate "fake data" by: randomly assigning within a domain
        self.area = None                                                    # generate "fake data" by: applying a grid over the osm map and clustering buildings within a grid to a sub area 
        self.sub_area = None                                                # generate "fake data" by: clustering sub-groups

# Define lists of possible structure types and functionalities
damage_states = ["DS1", "DS2", "DS3", "DS4"]
structure_types = ["S1L", "S1M", "S1H", "S2L", "S2M", "S2H", "S3", "S4L", "S4M", "S4H","S5L", "S5M", "S5H","C1L", "C1M", "C1H", "C2L", "C2M", "C2H", "C3L", "C3M", "C3H" "PC1","PC1L", "PC1M", "PC1H", "RM1L", "RM1M", "RM2L", "RM2M", "RM2H", "URML", "URMM", "MH"]
functionalities = ["Residential", "Commercial", "Hospital", "School", "Store"]


# add a building entry to the building list
building_1 = Building(1, 110100, 110100, "DS4", "S1L", "Residential",2, 30, 9.0, 3, 1972, 30)
building_2 = Building(2, 110111, 110111, "DS1", "RC", "Hospital", 5, 30, 9.0, 3, 1972)

# add buildings to sub areas
sub_area_1 = Sub_Area(1, "sub_area_1")
sub_area_1.add_building(building_1)

sub_area_2 = Sub_Area(2, "sub_area_2")
sub_area_2.add_building(building_2)

# add sub areas to areas
area_1 = Area(1, "area_1")
area_1.add_sub_area(sub_area_1)
area_1.add_sub_area(sub_area_2)

# Create 200 buildings
for i in range(3, 200):  # Start from 3 to avoid conflicts with existing building IDs
    building = Building(
        i, 
        110100 + i,
        110100 + i, 
        random.choices(damage_states), 
        random.choice(structure_types), 
        random.choice(functionalities), 
        random.uniform(1, 10), 
        random.uniform(1, 10),
        random.uniform(5.0, 15.0),
        i,
        random.randint(1950, 2020), 
    )
    # For demonstration purposes, alternate between sub-areas 1 and 2
    sub_area = sub_area_1 if i % 2 == 0 else sub_area_2
    sub_area.add_building(building)


data = [
    {
        "building_id": building.building_id,
        # "geometry": building.geometry,
        "x_coordinate": building.x_coordinate,
        "y_coordinate": building.y_coordinate,
        "structure_type": building.structure_type,
        "functionality": building.functionality,
        "plan_area": building.plan_area,
        "height": building.height,
        "age": building.age,
        "sub_area_id": sub_area.sub_area_id,
        "area_id": area_1.area_id
    }
    for sub_area in area_1.sub_areas
    for building in sub_area.buildings
]

# Create the DataFrame
df = pd.DataFrame(data)
print(df)

TypeError: __init__() missing 1 required positional argument: 'age'